In [1]:
import pandas as pd
import numpy as np
import numpy_financial as npf
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
from numpy import *

from ggplot import *
from scipy.stats import logistic
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import *
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz
import graphviz
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVC, SVR
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler


from linearmodels.iv import IV2SLS
from scipy import stats
from scipy.stats import trim_mean #trimmed mean
from scipy.stats import * #chi2_contingency로 카이제곱통계량계산
from scipy.stats import norm

import keyword
import random

#open api 불러올 때 필요한 것
from urllib.request import urlopen #request package 설치
from bs4 import BeautifulSoup #bs4 package 설치

# 그래프에서 한글 깨지지 않기 위헤 사용함
from matplotlib import rcParams
from matplotlib import font_manager,rc

rcParams['axes.unicode_minus']=False
plt.rcParams['font.family']='Malgun Gothic'
font_name=font_manager.FontProperties(fname='c:/Windows/Fonts/malgun.ttf').get_name()
rc('font',family=font_name)

import FinanceDataReader as fdr
import plotly.graph_objects as go
import plotly.subplots as ms
import math

import tensorflow as tf

import qgrid

In [2]:
# ln11 다중선형회귀분석

In [3]:
df=pd.read_csv('c:/temp/education2020.csv')
df.head()

,Unnamed: 0,year,metro,id,sex,age,number,education,marriage,asset,debt,income,income_d,industry,job,house,education_year
0,0,2020,G1,10000112,1,34,3,6,2,112000,54500,6593,4599,F,3,2,16
1,1,2020,G1,10000132,2,45,2,8,2,42500,17500,17720,15257,J,2,3,21
2,2,2020,G1,10000162,2,73,1,2,3,5712,0,908,725,T,4,2,6
3,3,2020,G1,10000182,1,58,2,4,2,14870,0,2748,2431,C,5,2,12
4,4,2020,G1,10000192,2,27,1,4,1,814,0,1015,893,R,2,3,12


In [4]:
df1=df[['income','age','education_year']] # 필요한 변수만 꺼내기

In [5]:
# 첫번째 방법: ols 함수 이용하기

In [7]:
# ols
edu_model=ols('income~age+education_year',data=df1).fit() # x 변수끼리는 + 로 연결한다
print(edu_model.summary())

                            OLS Regression Results                            
Dep. Variable:                 income   R-squared:                       0.164
Model:                            OLS   Adj. R-squared:                  0.164
Method:                 Least Squares   F-statistic:                     1773.
Date:                Wed, 19 Jun 2024   Prob (F-statistic):               0.00
Time:                        06:03:40   Log-Likelihood:            -1.7980e+05
No. Observations:               18064   AIC:                         3.596e+05
Df Residuals:                   18061   BIC:                         3.596e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept        322.1142    277.944      1.

In [8]:
# age coef: -9.7072: 마이너스! : 나이가 많을수록 소득이 줄어든다

In [9]:
# 2번째 방법: 실제로 행렬 계산해서 계수 추정하기

In [10]:
# 불러오기
df=pd.read_csv('c:/temp/education2020.csv')


# 전처리

# x변수들
edu=df1[['age','education_year']]
X=edu.values #(.values: 앞이 행렬로 바뀐다)

# 상수항
ones=np.ones(X.shape[0]) # 상수항 만들기
# X.shape 하면은 행렬 어케 구성된지 나옴 = 행렬의 dimension
# X.shape[0]: X.shape 했을 때 나오는 결과 값 (,)의 앞에거 가져오는 것
# 즉 ones= ~~ 로는 1,1,1,1,... 행렬을 만드는 것

# 상수항, x 변수 2개 으로 이루어진 행렬
X_mat=np.column_stack((ones,X))

# X_mat.shape 을 해보면 (row 개수, 칼럼 개수=3=상수항,x변수2개) 이렇게 나온다

# y변수
Y=df1['income'].values #(.values: 앞이 행렬로 바뀐다)


# 베타햇 추정하기 <- 이게 중요
beta_hat=np.linalg.inv(X_mat.T @X_mat)@(X_mat.T @Y)
#np.linal.inv : 역행렬로 하는 것
# .T : transpose '


# 추정한 Y = y_hat
Y_fitted=X_mat @ beta_hat



# std err=se(beta_hat) 계산해보기
# 그래야 뒤에 t, p-value 나옴

# 잔차
e_hat=Y-Y_fitted
n=len(e_hat)
# 시그마를 구해야 표준오차 구할 수 있다. 
sigma_hat2=(e_hat.T @ e_hat)/(n-3) # n-3= n-(x변수 개수+1)

XX_inv=np.linalg.inv(X_mat.T @ X_mat)

# XX_inv랑 sigma_hat2로 표준오차=se(beta_hat) 구할 수 있다.
# var(beta_hat)
var_beta_hat=sigma_hat2*XX_inv

# se(beta_hat)
np.sqrt(var_beta_hat)

array([[277.94376121,          nan,          nan],
       [         nan,   3.16994497,   4.46648209],
       [         nan,   4.46648209,  10.60068714]])

In [11]:
beta_hat

array([322.11417475,  -9.70719755, 488.05651908])

In [12]:
# 0522 시작

In [13]:
# 제곱항과 더미변수 추가

In [14]:
# 제곱항
# 제곱항 모형, 데이터프레임에 포함시키기
df1['age2']=df1['age']**2

# ols
edu_model1=ols('income~education_year+age+age2',data=df1).fit()
print(edu_model1.summary())

                            OLS Regression Results                            
Dep. Variable:                 income   R-squared:                       0.187
Model:                            OLS   Adj. R-squared:                  0.187
Method:                 Least Squares   F-statistic:                     1387.
Date:                Wed, 19 Jun 2024   Prob (F-statistic):               0.00
Time:                        06:28:28   Log-Likelihood:            -1.7954e+05
No. Observations:               18064   AIC:                         3.591e+05
Df Residuals:                   18060   BIC:                         3.591e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept      -9472.0780    511.480    -18.

In [16]:
# 위 문제 행렬로 coef 구해보기

In [17]:
# 불러오기
df=pd.read_csv('c:/temp/education2020.csv')
df1=df[['income','age','education_year']]
df1['age2']=df1['age']**2

# 전처리
edu=df1[['age','age2','education_year']]
X=edu.values 
ones=np.ones(X.shape[0])
X_mat=np.column_stack((ones,X))
Y=df1['income'].values

# 베타햇 추정하기 <- 이게 중요
beta_hat=np.linalg.inv(X_mat.T @X_mat)@(X_mat.T @Y)
#np.linal.inv : 역행렬로 하는 것
# .T : transpose '
beta_hat

array([-9.47207797e+03,  3.80629899e+02, -3.43928289e+00,  4.43657211e+02])

In [ ]:
# 더미변수

In [20]:
# 불러오기
df=pd.read_csv('c:/temp/education2020.csv')

# 전처리 1
df1=df[['income','age','education_year']]

In [21]:
# 더미변수 추가하기 <- 데이터 전처리 2

# 더미변수 1: sex_dummies
# 지금 df1에 성별 sex 변수있음
# 이걸 regression에 사용하기 위해 '더미변수'로 만들자
# sex_dum1, sex_dum2
#sex_dummies=pd.get_dummies(df1['sex']).rename(columns=lambda x: 'sex'+str(x))
# 그럼 알아서 더미 두개로 만들어진다
# .rename ~~ : 칼럼이름 지정
# lambda x: 람다 함수를 사용한다... 뭔소리야
#sex_dummies
# 1: 남자, 0: 여자
sex_dummies=pd.get_dummies(df['sex'],prefix='s')

# 더미변수 2: job_dummies
df=df.loc[df['job']!='A']
job_dummies=pd.get_dummies(df['job'],prefix='j')

# df2와 더미변수를 합친 데이터프레임을 만든다
# axis=1 : 칼럼 방향, 옆으로 붙이자
df2=pd.concat([df1,sex_dummies,job_dummies],axis=1)

In [22]:
# ols 하기

# 1: 그냥 ols
edu_model2=ols('income~age+education_year',data=df2).fit()
print(edu_model2.summary())

                            OLS Regression Results                            
Dep. Variable:                 income   R-squared:                       0.164
Model:                            OLS   Adj. R-squared:                  0.164
Method:                 Least Squares   F-statistic:                     1767.
Date:                Wed, 19 Jun 2024   Prob (F-statistic):               0.00
Time:                        06:47:23   Log-Likelihood:            -1.7955e+05
No. Observations:               18039   AIC:                         3.591e+05
Df Residuals:                   18036   BIC:                         3.591e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept        322.8971    278.183      1.

In [23]:
# 2: 더미변수 포함한 ols
# 더미변수는 하나씩 빼준다
edu_model3=ols('income~age+education_year+s_1+j_2+j_3+j_4+j_5+j_6+j_7+j_8+j_9',data=df2).fit()
print(edu_model3.summary())

                            OLS Regression Results                            
Dep. Variable:                 income   R-squared:                       0.228
Model:                            OLS   Adj. R-squared:                  0.228
Method:                 Least Squares   F-statistic:                     484.8
Date:                Wed, 19 Jun 2024   Prob (F-statistic):               0.00
Time:                        06:47:45   Log-Likelihood:            -1.7883e+05
No. Observations:               18039   AIC:                         3.577e+05
Df Residuals:                   18027   BIC:                         3.578e+05
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept      -2686.6412    304.369     -8.

In [24]:
# 해석
# s_1 = 남자: 1971 (양수)
# 남자가 여자(빠진 더미)보다 돈 더 많이 번다

# j_2: 4115
# j_1 보다 4115만큼 돈을 더 벌고 있구나